In [25]:
#import dependencies
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import sqlite3

In [26]:
#read in abortion data by state
#data ends in 2017
abortion_facts = pd.read_csv('GuttmacherDataCenter.csv')

In [27]:
#abortion_facts.columns

In [28]:
#rename column "summary" to "shootings"
abortion_facts = abortion_facts.rename(columns={"U.S. State":"State"})

In [29]:
#abortion_trial = abortion_facts[['State','% of all U.S. abortions, by state of occurrence, 2017']]
#abortion_trial

In [30]:
# Set option to display all rows
pd.set_option('display.max_rows', None)

In [31]:
#read in abortion data by state
#data ends in 2017
abortion_state = pd.read_csv('Guttmacher_Webscrape.csv', encoding='unicode_escape')

In [32]:
#abortion_state.head()

In [33]:
#do an inner join to merge data frames on year and state, so that the law total is added only per state and year with a mass shooting
df_abortion = pd.merge(abortion_facts, abortion_state, how="inner",on=["State"])
#df_abortion["No. of women who likely need public support for contraceptive s"]
#df_abortion.columns

In [34]:
df_abortion.columns=df_abortion.columns.str.replace(","," ")
df_abortion.head()

,State,% change in abortion rate (the no. of abortions per 1 000 women aged 15-44) by state of occurrence 2014-2017,% change in the no. of abortion clinics 2014-2017 [2],% change in the no. of abortion providers 2014-2017 [2],% change in women who likely need public support for contraceptive services and supplies 2016,% of all U.S. abortions by state of occurrence 2017,% of women aged 15-44 covered by Medicaid 2017,% of women aged 15-44 covered by private insurance 2017,% of women aged 15-44 who are uninsured 2017,% uninsured women aged 20-44 and <138% of the federal poverty level who likely need public support for contraceptive services and supplies 2016,...,% WORA: 25-29 Years,% WORA: 30-39 Years,% WORA: 40-49 Years,Avg. Distance (mi): 6 Weeks,Avg. Distance (mi): 12 Weeks,Avg. Distance (mi): 14 Weeks,Avg. Distance (mi): 20 Weeks,Avg. Distance (mi): 22 Weeks,Avg. Distance (mi): 24 Weeks,Avg. Distance (mi): 24+ Weeks
0,Alabama,-23,0,-22,10,0.7,18,63,14,28,...,15,29,28,NaN,115.0,118.0,137.0,179.0,474.0,507.0
1,Alaska,-14,33,-25,6,0.1,24,49,15,31,...,18,33,26,NaN,9.0,11.0,NaN,NaN,NaN,NaN
2,Arizona,-6,-11,-8,8,1.4,24,60,13,21,...,15,28,27,NaN,NaN,NaN,11.0,NaN,22.0,25.0
3,Arkansas,-30,0,0,13,0.4,28,58,11,20,...,15,29,28,NaN,NaN,NaN,160.0,328.0,NaN,360.0
4,California,-16,6,-18,2,15.4,27,62,9,18,...,16,30,27,NaN,4.0,8.0,9.0,12.0,NaN,14.0


In [35]:
df_abortion.columns = [x.replace('No. of women who likely need public support for contraceptive services and supplies','services needed:') for x in df_abortion.columns]

In [36]:
#df_abortion.columns

In [37]:
#establish the connection to postgresql
protocol = 'postgresql'
username = 'postgres'
password = 'admin'
host = 'localhost'
port = 5432
database_name = 'abortion_db_sql'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

In [17]:
#create table in sql
#check table exists
engine.table_names()

C:\Users\US11338\AppData\Local\Temp/ipykernel_18052/1858051120.py:3: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


[]

In [38]:
conn = sqlite3.connect("abortion.db")

In [43]:
#upload dataframes into sql
df_abortion.to_sql(name='df_abortion_sql', con=conn, if_exists='replace', index=False)

C:\Users\US11338\Anaconda3\lib\site-packages\pandas\core\generic.py:2872: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [44]:
abortion = pd.read_sql_query("select * from df_abortion_sql", con=conn)

In [46]:
abortion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 77 columns):
 #   Column                                                                                                                                                               Non-Null Count  Dtype  
---  ------                                                                                                                                                               --------------  -----  
 0   State                                                                                                                                                                50 non-null     object 
 1   % change in abortion rate (the no. of abortions per 1 000 women aged 15-44)  by state of occurrence  2014-2017                                                       50 non-null     int64  
 2   % change in the no. of abortion clinics  2014-2017 [2]                                                              